In [9]:
from dotenv import load_dotenv
_ = load_dotenv()

In [29]:
import os

print(os.environ["LANGCHAIN_API_KEY"])

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


# Manual Written Agent without binding to tools

In [38]:
import operator
from typing import Annotated, Literal, Union
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain_groq import ChatGroq
import os

# Create the LLM model and TavilySearchResults
llm = ChatGroq(model="llama3-70b-8192")
tavily_search = TavilySearchResults(max_results=3)

class State(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    mode: Literal["llm", "search", "NA"]

# Define the function to determine the mode
def determine_mode(state: State) -> State:
    print("Messages in determine mode function:")
    for msg in state['messages']:
        print(f"{msg.type}: {msg.content}")
    print("-------------------")
    conversation_history = "\n".join([f"{msg.type}: {msg.content}" for msg in state['messages']])
    latest_query = state["messages"][-1].content
    
    prompt = f"""Based on the conversation history and the latest user query below, determine if the query can be answered by LLM using the context from the conversation, or if it really needs an external web search to gather new information to answer the query. Try to not use web search unless necessary.
    
    Respond with ONLY "llm" or "search" without any additional text or explanation.
    
    Conversation history:
    {conversation_history}
    
    Latest query: {latest_query}

    Response (ONLY "llm" or "search"):"""

    response = llm.invoke(prompt)

    print("Response generated:", response.content.strip().lower())

    if "search" in response.content.strip().lower():
        return {"mode": "search"}
    return {"mode": "llm"}

def llm_response(state: State) -> State:
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response], "mode": "NA"}

def search_response(state: State) -> State:
    query = state["messages"][-1].content
    search_result = tavily_search.invoke(query)

    prompt = f"You are an AI assistant. Use the following search results to answer the user's question:\n{search_result}\n\nUser query: {query}"

    response = llm.invoke(prompt)
    return {"messages": [response], "mode": "NA"}

# Create the graph
graph = StateGraph(State)

# Add nodes
graph.add_node("determine_mode", determine_mode)
graph.add_node("llm_response", llm_response)
graph.add_node("search_response", search_response)

# Add edges
graph.set_entry_point("determine_mode")
graph.add_conditional_edges(
    "determine_mode",
    lambda x: x["mode"],
    {
        "llm": "llm_response",
        "search": "search_response"
    }
)
graph.add_edge("llm_response", END)
graph.add_edge("search_response", END)

# Compile the graph
chain = graph.compile()

# Initialize an empty conversation history
conversation_history = []

# Function to add a new message and get a response
def get_response(user_input):
    # Add the new user message to the conversation history
    conversation_history.append(HumanMessage(content=user_input))
    
    # Invoke the chain with the full conversation history
    result = chain.invoke({"messages": conversation_history})
    
    # Add the AI's response to the conversation history
    conversation_history.append(result['messages'][-1])
    
    print("Response:", result['messages'][-1].content)

# Example usage
get_response("Tell me a joke")
get_response("What was the last thing we talked about?")


Messages in determine mode function:
human: Tell me a joke
-------------------
Response generated: llm
Response: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!
Messages in determine mode function:
human: Tell me a joke
ai: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!
human: What was the last thing we talked about?
-------------------
Response generated: llm
Response: We talked about a joke! I shared a bicycle-themed joke with you, and it was a bit of a "tire"-d pun.


In [36]:
get_response("explain it to me as if I am 5 year old")

Messages in determine mode function:
human: Tell me a joke
ai: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!
human: What was the last thing we talked about?
ai: We talked about a joke! I shared a joke with you about a bicycle being two-tired.
human: explain it to me as if I am 5 year old
-------------------
Response generated: llm
Response: OH BOY!

So, you know how bicycles have wheels, right? Like, round things that make the bike move?

Well, the joke is saying that the bicycle is "two-tired". Get it? "Two-tired" instead of "too tired"!

It's like saying the bike is tired, but it's also making a funny pun on the word "tired" because it has TWO TIRES! 

So, the joke is saying that the bike can't stand up by itself because it's "two-tired"... it's a play on words!


In [37]:
get_response("thats so cool. how can i start my own bank?")

Messages in determine mode function:
human: Tell me a joke
ai: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you laugh!
human: What was the last thing we talked about?
ai: We talked about a joke! I shared a joke with you about a bicycle being two-tired.
human: explain it to me as if I am 5 year old
ai: OH BOY!

So, you know how bicycles have wheels, right? Like, round things that make the bike move?

Well, the joke is saying that the bicycle is "two-tired". Get it? "Two-tired" instead of "too tired"!

It's like saying the bike is tired, but it's also making a funny pun on the word "tired" because it has TWO TIRES! 

So, the joke is saying that the bike can't stand up by itself because it's "two-tired"... it's a play on words!
human: thats so cool. how can i start my own bank?
-------------------
Response generated: search
Response: Starting your own bank! That's quite an ambitious goal! According to the search res

# Postgres integrated Langgraph Agent with partial Langsmith tracing

In [ ]:
import operator
from typing import Annotated, Literal, Union
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain_groq import ChatGroq
import os
from psycopg_pool import AsyncConnectionPool
from psycopg.rows import dict_row
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
import asyncio
import sys

from langsmith import Client
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager, AsyncCallbackManager
from langsmith import trace

# Set up LangSmith
client = Client(api_url=os.environ["LANGSMITH_ENDPOINT"], api_key=os.environ["LANGSMITH_API_KEY"])
tracer = LangChainTracer(project_name=os.environ["LANGSMITH_PROJECT"])
callback_manager = AsyncCallbackManager([tracer])

# Modify your LLM creation to include callbacks
llm = ChatGroq(model="llama3-70b-8192", callbacks=[tracer])
tavily_search = TavilySearchResults(max_results=3)

class State(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    mode: Literal["llm", "search", "NA"]

# Define the function to determine the mode
def determine_mode(state: State) -> State:
    with trace("determine_mode") as run:
        print("Messages in determine mode function:")
        for msg in state['messages']:
            print(f"{msg.type}: {msg.content}")
        print("-------------------")
        conversation_history = "\n".join([f"{msg.type}: {msg.content}" for msg in state['messages']])
        latest_query = state["messages"][-1].content
        
        prompt = f"""Based on the conversation history and the latest user query below, determine if the query can be answered by LLM using the context from the conversation, or if it really needs an external web search to gather new information to answer the query. Try to not use web search unless necessary.
        
        Respond with ONLY "llm" or "search" without any additional text or explanation.
        
        Conversation history:
        {conversation_history}
        
        Latest query: {latest_query}

        Response (ONLY "llm" or "search"):"""

        response = llm.invoke(prompt)

        print("Response generated:", response.content.strip().lower())

        if "search" in response.content.strip().lower():
            run.end(outputs={"mode": "search"})
            return {"mode": "search"}
        run.end(outputs={"mode": "llm"})
        return {"mode": "llm"}

def llm_response(state: State) -> State:
    with trace("llm_response") as run:
        messages = state["messages"]
        response = llm.invoke(messages)
        run.end(outputs={"messages": [response], "mode": "NA"})
        return {"messages": [response], "mode": "NA"}

def search_response(state: State) -> State:
    with trace("search_response") as run:
        query = state["messages"][-1].content
        search_result = tavily_search.invoke(query)

        prompt = f"You are an AI assistant. Use the following search results to answer the user's question:\n{search_result}\n\nUser query: {query}"

        response = llm.invoke(prompt)
        run.end(outputs={"messages": [response], "mode": "NA"})
        return {"messages": [response], "mode": "NA"}

def create_agent(checkpointer):
    # Create the graph
    graph = StateGraph(State)

    # Add nodes
    graph.add_node("determine_mode", determine_mode)
    graph.add_node("llm_response", llm_response)
    graph.add_node("search_response", search_response)

    # Add edges
    graph.set_entry_point("determine_mode")
    graph.add_conditional_edges(
        "determine_mode",
        lambda x: x["mode"],
        {
            "llm": "llm_response",
            "search": "search_response"
        }
    )
    graph.add_edge("llm_response", END)
    graph.add_edge("search_response", END)

    # Compile the graph
    chain = graph.compile(checkpointer=checkpointer)
    return chain

async def setup_pool():
    return AsyncConnectionPool(
        conninfo=os.getenv("DATABASE_URL"),
        max_size=20,
        kwargs={
            "autocommit": True,
            "prepare_threshold": 0,
            "row_factory": dict_row,
        },
    )


# Function to add a new message and get a response
async def get_response(pool, user_input):
    async with pool.connection() as conn:
        try:
            # Initialize persistent chat memory
            memory = AsyncPostgresSaver(conn)

            # IMPORTANT: You need to call .setup() the first time you're using your memory
            await memory.setup()

            # Create a LangGraph agent
            langgraph_agent = create_agent(checkpointer=memory)

            with trace("LangGraph Agent") as parent_run:
                response = await langgraph_agent.ainvoke(
                    {"messages": [HumanMessage(content=user_input)]},
                    {"configurable": {"thread_id": "5"}}
                )
            
            # Process the chunks from the agent
            return response['messages'][-1].content
        except Exception as e:
            print(f"An error occurred: {e}")
            raise


async with await setup_pool() as pool:
    response = await get_response(pool, "give one word reply for this: hi")
    print("Response:", response)

Messages in determine mode function:
human: give one word reply for this: hi
ai: Hello
human: give one word reply for this: hi
ai: Hey
human: give one word reply for this: hi
ai: Hi
human: give one word reply for this: hi
-------------------
Response generated: llm
Response: Hey


# Experiment for langsmith full tracing

In [ ]:

import operator
from typing import Annotated, Literal, Union
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain_groq import ChatGroq
import os
from psycopg_pool import AsyncConnectionPool
from psycopg.rows import dict_row
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langgraph.checkpoint.postgres import PostgresSaver

import asyncio
import sys
from psycopg_pool import ConnectionPool

from langsmith import Client
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager, AsyncCallbackManager
from langsmith import trace
import cProfile


# Set up LangSmith
client = Client(api_url=os.environ["LANGSMITH_ENDPOINT"], api_key=os.environ["LANGSMITH_API_KEY"])
tracer = LangChainTracer(project_name=os.environ["LANGSMITH_PROJECT"])
callback_manager = CallbackManager([tracer])

tavily_search = TavilySearchResults(max_results=3)

tools = [tavily_search]

tool_node = ToolNode(tools)

# Modify your LLM creation to include callbacks
model = ChatGroq(model="llama3-70b-8192").bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    mode: Literal["llm", "search", "NA"]

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

def call_model(state: MessagesState):
    messages = state['messages']
    system_message = SystemMessage(content="You are an AI assistant with access to tools. When you need more information to answer a question, use the appropriate tool. Always use tools for current events or data that might change.")
    messages = [system_message] + messages

    response = model.invoke(messages)
    print("Call model's response:", response)
    return {"messages": [response]}

def create_agent(checkpointer):
    # Create the graph
    graph = StateGraph(MessagesState)

    graph.add_node("agent", call_model)
    graph.add_node("tools", tool_node)

    graph.add_edge("__start__", "agent")
    graph.add_conditional_edges(
        "agent",
        should_continue,
    )
    graph.add_edge("tools", 'agent')


    # Compile the graph
    chain = graph.compile(checkpointer=checkpointer)
    return chain

def setup_pool():
    return ConnectionPool(
        conninfo=os.getenv("DATABASE_URL"),
        max_size=20,
        kwargs={
            "autocommit": True,
            "prepare_threshold": 0,
            "row_factory": dict_row,
        },
    )



# # Function to add a new message and get a response
# async def get_response(pool, user_input):
#     async with pool.connection() as conn:
#         try:
#             # Initialize persistent chat memory
#             memory = AsyncPostgresSaver(conn)

#             # IMPORTANT: You need to call .setup() the first time you're using your memory
#             await memory.setup()

#             # Create a LangGraph agent
#             langgraph_agent = create_agent(checkpointer=memory).with_config(callback_manager=callback_manager)

#             response = await langgraph_agent.ainvoke(
#                 {"messages": [HumanMessage(content=user_input)]},
#                 {"configurable": {"thread_id": "7"},
#                  "callbacks": callback_manager}
#             )

#             # Process the chunks from the agent
#             return response['messages'][-1].content
#         except Exception as e:
#             print(f"An error occurred: {e}")
#             raise

# async with await setup_pool() as pool:
#     response = await get_response(pool, "who is the president of the maldives?")
#     print("Response:", response)




# langgraph_agent = create_agent()

# response = langgraph_agent.invoke(
#     {"messages": [HumanMessage(content="who is the president of the united states?")]},
#     {"configurable": {"thread_id": "7"}}
# )

# # Process the chunks from the agent
# print(response['messages'][-1].content)



def get_response(memory, user_input):
    try:
        langgraph_agent = create_agent(checkpointer=memory)
        response = langgraph_agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            {"configurable": {"thread_id": "7"}}
        )
        return response['messages'][-1].content
    except Exception as e:
        print(f"An error occurred: {e}")
        raise


with setup_pool() as pool:
    with pool.connection() as conn:
        memory = PostgresSaver(conn)
        memory.setup()  # Synchronous setup
        response = get_response(memory, "who is the PM of israel?")
        print("Response:", response)



Call model's response: content='' additional_kwargs={'tool_calls': [{'id': 'call_jdjk', 'function': {'arguments': '{"query":"current prime minister of Israel"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 6777, 'total_tokens': 6821, 'completion_time': 0.127876143, 'prompt_time': 0.342764971, 'queue_time': 0.04144218500000002, 'total_time': 0.470641114}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-527c77fb-ce3d-4eec-b003-031638c27ec4-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current prime minister of Israel'}, 'id': 'call_jdjk', 'type': 'tool_call'}] usage_metadata={'input_tokens': 6777, 'output_tokens': 44, 'total_tokens': 6821}
Call model's response: content='The Prime Minister of Israel is Benjamin Netanyahu.' additional_kwargs={} response_metadata={'token_usage': {'completion